In [1]:
import pandas as pd
import numpy as np

from feature_functions import *

from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Variables:
future = 1

averageKeys = [2]
momentumKeys = [3, 4, 5, 8, 9, 10]
heikenKeys = [15]
EMAKeys = [2,10,20]
fourierKeys = [10,20,30]
sineKeys = [5,6]
wadlKey = [15]
stochasticKey = [3, 4, 5, 8, 9, 10]
williamsKey = [6, 7, 8, 9, 10]


# Loading the data
df = pd.read_csv("data/EURUSDday.csv")
df.columns = ['date','open','high','low','close','volume']
df.date = pd.to_datetime(df.date, format='%d.%m.%Y %H:%M:%S.%f')
df = df.set_index(df.date)
df = df[['open','high','low','close','volume']]

# Drop elements where there is no mouvement (Market closed) like in the weekends
df = df.drop_duplicates(keep=False)
df.head(2)

,open,high,low,close,volume
date,,,,,
2011-01-03 22:00:00,1.33596,1.34298,1.32923,1.33068,204936.7404
2011-01-04 22:00:00,1.33047,1.33248,1.31262,1.31490,191026.6300


In [2]:
# To concat the original dataframe with averages
df_with_features = df

# Add Averages
df_with_features = pd.concat([df_with_features, moving_averages(df, averageKeys)],axis=1)
# df_with_features = pd.concat([df_with_features, exponential_averages(df, EMAKeys)],axis=1)

# Add heikenashi
df_with_features = pd.concat([df_with_features, heinkenashi(df, heikenKeys)],axis=1)

# Add momentum
df_with_features = pd.concat([df_with_features, momentum(df, momentumKeys)],axis=1)

# Add fourier
df_with_features= pd.concat([df_with_features, fourier(df, fourierKeys)],axis=1)

# Add sine
df_with_features= pd.concat([df_with_features, sine(df, sineKeys)],axis=1)

# Add wadl
df_with_features= pd.concat([df_with_features, wadl(df, wadlKey)],axis=1)

# Add stochastic
df_with_features= pd.concat([df_with_features, stochastic(df, stochasticKey)],axis=1)

# Add williams
df_with_features= pd.concat([df_with_features, williams(df, williamsKey)],axis=1)

df_with_features['result'] = create_results(df , 'close', future)
df_with_features = df_with_features.dropna()
df_with_features

,open,high,low,close,volume,MA2 open,MA2 high,MA2 low,MA2 close,HA15 open,...,stochastics 9 K,stochastics 9 D,stochastics 10 K,stochastics 10 D,williams 6 R,williams 7 R,williams 8 R,williams 9 R,williams 10 R,result
date,,,,,,,,,,,,,,,,,,,,,
2011-02-14 22:00:00,1.34883,1.35504,1.34609,1.34866,191687.1815,1.349715,1.355430,1.344445,1.348745,1.355296,...,-2.912088,1.189885,-2.912088,1.189885,-104.315961,-103.237630,-102.912088,-102.912088,-102.912088,1.0
2011-02-15 22:00:00,1.34865,1.35879,1.34619,1.35680,206301.3507,1.348740,1.356915,1.346140,1.352730,1.352476,...,32.317636,7.952618,32.317636,7.952618,-55.527319,-55.527319,-64.699950,-67.682364,-67.682364,1.0
2011-02-16 22:00:00,1.35666,1.36183,1.35368,1.36087,199447.1311,1.352655,1.360310,1.349935,1.358835,1.352542,...,45.562279,24.989276,41.712835,23.706128,-42.598475,-42.598475,-42.598475,-54.437721,-58.287165,1.0
2011-02-17 22:00:00,1.36087,1.37144,1.35459,1.36921,200646.7903,1.358765,1.366635,1.354135,1.365040,1.355401,...,83.894536,53.924817,66.591024,46.873832,-16.105464,-16.105464,-16.105464,-16.105464,-33.408976,0.0
2011-02-20 22:00:00,1.36972,1.37147,1.36471,1.36752,120506.6307,1.365295,1.371455,1.359650,1.368365,1.359714,...,78.526048,69.327621,78.526048,62.276636,-18.630678,-21.473952,-21.473952,-21.473952,-21.473952,0.0
2011-02-21 22:00:00,1.36749,1.37030,1.35269,1.36497,238283.5475,1.368605,1.370885,1.358700,1.366245,1.364035,...,70.425667,77.615417,70.425667,71.847580,-22.590782,-27.024358,-29.574333,-29.574333,-29.574333,1.0
2011-02-22 22:00:00,1.36495,1.37863,1.36460,1.37479,205282.9413,1.366220,1.374465,1.358645,1.369880,1.363949,...,101.620076,83.523931,101.620076,83.523931,11.580049,11.580049,5.299539,1.620076,1.620076,1.0
2011-02-23 22:00:00,1.37482,1.38198,1.37040,1.37992,220566.5610,1.369885,1.380305,1.367500,1.377355,1.367346,...,122.185648,98.077131,117.916137,96.653960,33.293932,29.473317,29.473317,22.185648,17.916137,0.0
2011-02-24 22:00:00,1.37992,1.38377,1.37237,1.37529,180676.2400,1.377370,1.382875,1.371385,1.377605,1.372063,...,90.678203,104.827976,90.678203,103.404805,-10.295931,-10.264290,-9.321797,-9.321797,-9.321797,1.0


In [ ]:
# The data
interval = future + 1
run_length = 300
correct_predictions = 0
for i in range(run_length):
    X = df_with_features.iloc[:df_with_features.shape[0]- (i*interval)-future -1 ,:-1]
    y = df_with_features.result

    # Split the data
    
    X_train= X.iloc[:X.shape[0]-1,:]
    X_test= df_with_features.iloc[X.shape[0]+future:X.shape[0]+future+1,:-1]
    
# #     Scale the data
#     scaler = preprocessing.StandardScaler().fit(X_train)
#     X_train = scaler.transform(X_train) 
#     X_test = scaler.transform(X_test) 
    
    y_train= y[:X.shape[0]-1]
    y_test =y [X.shape[0]+future:X.shape[0]+future+1]

#     Model
#     clf = GradientBoostingClassifier(random_state=5, learning_rate=0.1, n_estimators=100)
    clf = SVR(gamma= 0.1, C=1.0, epsilon=0.2)
#     clf = SVC(gamma=0.1)
    clf.fit(X_train, y_train) 

    # Predictions
    if(clf.predict(X_test)[0]>0.5):
        predicted = [1.0]
    else :
        predicted = [0.0]
    correct_predictions += accuracy_score(y_test, predicted)
    if ((i+1)%20 == 0):
        print (str(accuracy_score(y_test, predicted)) +' ' + str(i+1) + ' accuracy: '+ str(correct_predictions/(i+1)))
    
    
accuracy = correct_predictions/run_length
accuracy

0.0 20 accuracy: 0.55
1.0 40 accuracy: 0.475
1.0 60 accuracy: 0.45
1.0 80 accuracy: 0.45
1.0 100 accuracy: 0.47
1.0 120 accuracy: 0.4666666666666667
0.0 140 accuracy: 0.4928571428571429
0.0 160 accuracy: 0.5125


In [ ]:
correct_predictions/run_length